This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_residuals_histogram(y_true, y_pred, savepath,
                             stats, title='residuals histogram', label='residuals'):
    """
    Method to calculate and plot the histogram of residuals from regression model

    Args:

        y_true: (numpy array), array of true y data values

        y_pred: (numpy array), array of predicted y data values

        savepath: (str), path to save the plotted precision-recall curve

        stats: (dict), dict of training or testing statistics for a particular run

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    #ax.set_title(title)
    # do the actual plotting
    residuals = y_true - y_pred

    #Output residuals data and stats to spreadsheet
    path = os.path.dirname(savepath)
    pd.DataFrame(residuals).describe().to_csv(os.path.join(path,'residual_statistics.csv'))
    pd.DataFrame(residuals).to_csv(path+'/'+'residuals.csv')

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=residuals)
    ax.hist(residuals, bins=num_bins, color='b', edgecolor='k')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plot_stats(fig, stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, pd.DataFrame(residuals).describe().to_dict()[0], x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([1.77283020e+02, 2.39214774e+02, 2.14185752e+02, 1.46594936e+02,
       1.40876460e+02, 9.96717956e+01, 9.45736257e+01, 9.62792101e+01,
       2.02290968e+02, 7.75902143e+01, 1.02665208e+02, 1.72420203e+02,
       1.22487016e+02, 1.78669886e+02, 1.87709915e+02, 1.79995444e+02,
       1.53899276e+02, 1.25776102e+02, 3.69780134e+02, 3.02985863e+02,
       3.19706165e+02, 2.29722452e+02, 2.27914069e+02, 1.72040539e+02,
       1.19298374e+02, 1.24022947e+02, 7.70450205e+01, 7.45855168e+01,
       1.74631224e+02, 1.31350152e+02, 4.50731986e+01, 1.09473901e+02,
       0.00000000e+00, 2.56639133e+01, 3.90426011e+02, 2.36177315e+02,
       8.39207969e+01, 1.56229730e+02, 7.39283454e+01, 6.86792014e+01,
       1.13135302e+02, 7.34978297e+01, 1.63297865e+02, 3.20094037e+01,
       0.00000000e+00, 6.94361418e+01, 9.24561079e+01, 1.56627930e+02,
       6.36890317e+01, 5.90096117e+00, 1.56215250e+01, 7.38023139e+01,
       3.57160729e+02, 2.91632073e+01, 1.49611167e+02, 3.07830539e+02,
       2.26412639e+01, 8.03088872e+01, 5.18644462e+01, 1.39599684e+02,
       1.51797441e+02, 2.89712978e+02, 1.18302658e+02, 2.94893175e+02,
       3.39191772e+01, 3.93315709e+01, 2.99490808e+01, 2.58389489e+01,
       0.00000000e+00, 0.00000000e+00, 4.51430137e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.70304392e+01, 0.00000000e+00,
       0.00000000e+00, 1.39370004e+02, 3.20782685e+01, 3.35683107e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.49408401e+01,
       7.74558848e+00, 6.05219124e+01, 3.21439514e+01, 1.61615856e+02,
       1.31304680e+02, 1.08979858e+02, 8.64067635e+01, 0.00000000e+00,
       0.00000000e+00, 1.28935952e+02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.44184734e+01, 6.21108106e+01, 8.83802997e+01, 2.41335092e+01,
       6.53557069e+01, 4.48563810e+01, 0.00000000e+00, 1.33902387e+01,
       6.12137026e+01, 4.85711869e+01, 4.91573257e+01, 8.68896540e+01,
       3.61795100e+01, 1.04542615e+02, 1.05693165e+02, 8.65356568e+01,
       8.04591080e+01, 8.51942082e+01, 1.00949225e+02, 1.26886312e+02,
       1.85404211e+02, 0.00000000e+00, 2.02289181e-01, 1.38892855e+02,
       1.37126059e+02, 1.26881899e+02, 1.70388617e+02, 8.98167378e+01,
       9.36717096e+01, 1.50078284e+01, 1.93409720e+01, 0.00000000e+00,
       1.24480773e+01, 4.70862505e+01, 1.27378189e+02, 1.14554644e+02,
       1.66187429e+02, 1.06702335e+02, 2.76340269e+02, 0.00000000e+00,
       4.84645306e+01, 2.94415703e+02, 1.65778433e+02, 1.39763133e+02,
       3.15202492e+01, 1.40538550e+02, 2.29064683e+02, 1.13901487e+02,
       4.80411705e+01, 2.53031123e+01, 3.81465155e+01, 6.49749186e+01,
       4.22355880e+01, 2.62303388e+01, 9.53324888e+01, 9.10665100e+00,
       0.00000000e+00, 1.05110827e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.84401941e+01, 5.54728596e+01, 5.13197137e+01,
       5.25266742e+01, 4.81837797e+01, 3.08078053e+01, 2.02393592e+02,
       5.65407126e+01, 3.81230146e+01, 5.46661575e+01, 5.87368740e+01,
       1.29976759e+02, 4.16645268e+01, 2.82496436e+01, 3.40111139e+01,
       0.00000000e+00, 6.50315409e+01, 1.89559087e+01, 3.92079232e+01,
       1.52429429e+02, 9.46538849e+01, 8.30269047e+01, 0.00000000e+00,
       4.00673869e+01, 2.50852147e+01, 3.11645320e+01, 2.75959946e+01,
       0.00000000e+00, 9.54966739e+01, 4.94252903e+01, 1.05442368e+02,
       0.00000000e+00, 0.00000000e+00, 6.89012756e+00, 0.00000000e+00,
       0.00000000e+00, 7.05592531e+00, 5.40607889e+01, 6.90074178e+01,
       0.00000000e+00, 0.00000000e+00, 1.29601532e+02, 1.55663046e+02,
       1.36431444e+02, 1.89093084e+02, 1.79654132e+02, 2.07904807e+02,
       2.34660651e+02, 3.36224697e+02, 9.15340044e+01, 1.16180977e+02,
       1.78939785e+02, 1.14272223e+02, 1.13869162e+02, 2.43428371e+02,
       1.14407716e+02, 1.86026530e+02, 1.30347188e+02, 1.26700822e+02,
       2.44148911e+02, 1.55694637e+02, 2.59886084e+02, 2.20161511e+02,
       1.96409134e+02, 1.44708713e+02, 1.91836674e+02, 1.41535230e+02,
       2.10335728e+02, 1.03165153e+02, 1.35390845e+02, 2.51196620e+02,
       2.37062344e+02, 2.73613179e+02, 3.62461054e+02, 1.77346156e+02,
       2.26507616e+02, 3.23822637e+02, 2.04046571e+02, 2.23816699e+02,
       3.40437655e+02, 2.55478983e+02, 3.18399908e+02, 5.98511381e+01,
       1.43517205e+02, 8.23163755e+01, 1.97124130e+02, 9.69915119e+01,
       2.10107835e+02, 1.61293351e+02, 9.69267962e+01, 9.83944608e+01,
       1.93969856e+02, 1.35408933e+02, 1.31977595e+02, 1.83449190e+02,
       1.02794584e+02, 1.60233988e+02, 3.64707101e+02, 1.86717789e+02,
       1.44889042e+02, 3.17205543e+02, 3.17356871e+02, 3.28541723e+02,
       3.23634151e+02, 3.46087908e+02, 1.20403530e+02, 1.41453455e+02,
       7.10448978e+01, 8.95779086e+01, 1.24842572e+02, 8.85969515e+01,
       1.38575932e+02, 1.36443088e+02, 1.47525298e+02, 6.40241718e+01,
       5.55045542e+01, 1.38091226e+02, 7.74665625e+01, 1.33883367e+02,
       1.32392433e+02, 1.26818170e+02, 5.68526124e+01, 1.84907725e+02,
       1.25571144e+02, 2.72871580e+02, 2.10329787e+02, 2.83042429e+02,
       2.27717761e+02, 2.47909595e+02, 2.42965465e+02, 2.00760493e+02,
       2.04542101e+02, 3.18785354e+02, 2.71906598e+02, 2.09325439e+02,
       1.35426489e+02, 8.10050964e+01, 5.74165434e+01, 1.04682468e+02,
       9.45208382e+01, 9.43913417e+01, 1.78159873e+02, 9.64735725e+01,
       2.72145096e+01, 6.07829098e+01, 2.42317220e+01, 4.96708469e+01,
       7.34595746e+01, 7.19683460e+01, 1.82785945e+01, 4.28892788e+01,
       3.27734114e+01, 1.35840083e+02, 1.02112315e+02, 1.73385595e+02,
       1.84215977e+02, 1.97176332e+02, 1.84779292e+02, 1.70292754e+02,
       1.63409286e+02, 1.63793911e+02, 1.11384136e+02, 1.54672899e+02,
       1.15407866e+02, 1.81387575e+02, 0.00000000e+00, 2.98870605e+01,
       7.73860277e+01, 9.14620807e+01, 8.35686242e+01, 9.06089707e+01,
       2.71319339e+02, 1.09488956e+02, 9.60360384e+01, 2.58828051e+02,
       1.17249168e+02, 1.37791901e+02, 1.85661694e+02, 1.12287386e+02,
       1.40364781e+02, 1.37369485e+02, 1.21692419e+02, 3.35321293e+02,
       9.37481720e+01, 2.63543371e+02, 4.21335070e+01, 3.63667394e+01,
       2.90221571e+02, 2.71330052e+01, 4.45796202e+00, 1.91667256e+01,
       9.53050436e+01, 3.73225941e+01, 1.12949106e+02, 5.04471564e+01,
       1.34244899e+02, 1.43075475e+02, 1.40737465e+02, 1.40488835e+02,
       1.04314376e+01, 1.43739984e+02, 1.61966268e+02, 1.78944559e+02,
       1.82449930e+02, 3.87496682e+01, 3.50703870e+01, 6.20947510e+01,
       6.92239663e+00, 0.00000000e+00, 0.00000000e+00, 6.23247673e+01,
       4.92192870e+01, 5.92192098e+01, 7.32347672e+01, 1.60334633e+02,
       5.19157130e+01, 2.84812922e+01, 2.83147240e+01, 2.55044144e+00,
       3.23684503e+01, 4.56348093e+01, 0.00000000e+00, 3.86517633e+01,
       2.99157066e+01, 7.07659236e+01, 2.67038101e+01, 3.37558922e+01,
       3.74680233e+01, 8.17268550e+01, 7.65032664e+01, 7.90798034e+01,
       7.72377141e+01, 6.23324679e+01, 5.22740325e+01, 1.21693503e+02,
       3.81925114e+01, 5.21415070e+01, 0.00000000e+00, 4.44093613e+01,
       4.39454900e+01, 3.91274706e+01, 1.90870878e+02, 1.66347669e+02,
       8.65192126e+01, 8.25826337e+01, 1.77019953e+01, 1.05624926e+02,
       3.62234796e+01, 1.42323884e+02, 3.48683785e+01, 4.63580983e+01,
       3.07458097e+01, 5.78409146e+01, 4.67667976e+01, 3.53717988e+01,
       5.24254612e+01, 4.66982172e+01, 2.28817925e+01, 8.97135102e+01,
       7.80670028e+01, 2.19120080e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.88957798e+01, 0.00000000e+00,
       2.38090660e+01])
y_pred = array([ 2.46824268e-005,  1.48720127e-005,  1.25837471e-270,
        8.78222560e-135,  3.28018776e-133,  2.06042835e-231,
        7.94081189e-028,  3.97993955e-192,  4.30335593e-231,
        1.98940071e-027,  3.39871916e-231,  0.00000000e+000,
        0.00000000e+000,  2.74497073e-130,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        3.24338957e-010,  5.14599817e-236,  4.78919061e-236,
        3.07381952e-304,  0.00000000e+000,  0.00000000e+000,
        2.10224626e-032,  8.42027393e-148,  1.48633439e-032,
        0.00000000e+000,  1.05517298e-252,  1.59223042e-036,
        2.95120309e-242,  3.10265696e-029,  6.58045386e-003,
        4.27152655e-038,  9.20767117e-137,  9.92729010e-111,
        3.71369616e-025,  1.08192522e-003,  3.56856916e-022,
        2.13492880e-021,  1.84375346e-010,  1.27840792e-013,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        1.40573846e-038,  0.00000000e+000,  0.00000000e+000,
        1.89981767e-098,  1.35588136e-075,  0.00000000e+000,
        8.21371737e-027,  0.00000000e+000,  1.57577633e-004,
        1.04022784e-020,  1.31274761e-036,  2.22705163e-005,
        0.00000000e+000,  3.50877210e-011,  5.30490469e-024,
        8.64306811e+000,  0.00000000e+000,  1.37469631e-005,
        2.18213044e-009,  7.07612205e-001,  9.92572461e-001,
        5.38578288e-002,  5.44776787e-249,  0.00000000e+000,
       -7.18870662e-239,  4.73036284e-238,  5.89732765e-239,
        5.31358639e-006, -9.22853000e-009, -2.76076032e-005,
        5.48357800e-004,  4.94245387e-251,  5.63977402e-095,
        9.57739162e-011,  1.85675892e-012,  2.83541773e-007,
        3.31623371e-242,  2.65759088e-242,  8.29806067e-008,
        2.58578678e-007,  2.03445047e-010,  2.87365010e-072,
        1.31411899e-011,  2.29517165e+000,  1.24751592e-020,
        9.04714408e-009,  0.00000000e+000,  0.00000000e+000,
        5.09400716e-248,  6.87664887e-014,  5.26793332e-238,
       -4.69803534e-258,  5.58162334e-240,  3.64212853e-255,
        2.62492659e-244,  3.26018949e-104,  3.83666377e-012,
        6.91097764e-009,  8.19500489e-013,  1.99731957e-003,
        1.71695051e-010,  0.00000000e+000,  1.49563477e-006,
        8.20419406e-003,  2.42091093e+000,  2.24321298e-007,
        3.09134622e-034,  9.46572441e-015,  7.54515501e-027,
        0.00000000e+000,  1.35750262e-162,  9.73511720e-287,
        0.00000000e+000,  1.03312794e-022,  3.31692117e-011,
        4.66390953e-005,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  3.05833775e-025,  1.00098061e-022,
        0.00000000e+000,  4.49083689e+001,  8.85411804e+000,
        0.00000000e+000,  0.00000000e+000,  0.00000000e+000,
        0.00000000e+000,  9.63009969e-014,  7.45936681e-023,
        2.02896282e-110,  2.63545356e-048,  8.70887330e-023,
        4.33126605e-005,  0.00000000e+000,  3.54159428e-008,
        1.47812937e-028,  9.31103473e-003,  3.28189038e-013,
        3.10207527e-098,  3.30773736e-047,  1.23908030e-079,
        4.87473896e-088,  1.74271768e-006,  6.30098276e+000,
        8.86925717e-237,  2.23118754e-234,  8.77518228e-001,
        5.63770956e-002,  1.77540981e-008,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000, -5.73417739e-011,
        0.00000000e+000, -3.33018493e-017,  3.24803541e-252,
        1.23272672e-101,  1.75755223e-006,  6.79515670e-008,
        7.91180825e-004,  3.05088379e-003,  1.39148106e-076,
        1.29362111e-006,  7.17100678e-006,  1.53778008e-069,
        1.69911835e-030,  4.29464043e-051,  7.18690202e-004,
        0.00000000e+000,  2.50949562e-239,  0.00000000e+000,
        4.70761658e-033,  6.87568961e-021,  1.23612033e-253,
        1.99852667e-076,  3.48342036e-239,  3.86094403e-239,
        1.14522347e-004,  6.40976246e-008,  6.14916987e+000,
        2.54432688e-017,  4.10574879e-017,  0.00000000e+000,
        2.00379770e-006,  1.10932723e-063,  1.74874865e-004,
        5.63306841e-013,  1.68858898e-198,  2.40196087e-198,
        0.00000000e+000,  2.80827713e-199,  2.39317091e-198,
        1.60942093e-003,  7.03817567e-032,  5.43277992e-204,
        0.00000000e+000,  2.73855791e-001,  3.52339917e-008,
        0.00000000e+000,  4.68835581e+000,  2.55638927e-001,
        6.39647127e-020,  4.63950126e-008,  8.15440091e+000,
        2.04901185e-001,  2.85846476e-001,  2.24448289e-005,
        8.22187280e+000,  3.16552976e+000,  6.45845271e-010,
        2.77804681e+000,  4.75718022e+000,  3.19311664e+000,
        2.50888735e-001,  2.87503449e-001,  3.62198440e-001,
        6.04152957e-001,  5.52259807e-002,  1.75712630e-020,
        1.23958296e-020,  2.08197484e-008,  1.46857067e-008,
        5.14129506e+000,  2.63042658e+000,  2.70136542e+000,
        5.14886102e+000,  5.01701368e-001,  5.67465023e-001,
        7.79460020e-001,  4.78377799e-002,  5.42865052e-002,
        8.93722558e+000,  6.37583699e+000,  6.86364339e+000,
        1.08857147e-010,  5.20155306e+000,  6.24440883e-004,
        3.13322578e+000,  4.21558239e+000,  1.73688711e-020,
        3.47558148e-001,  1.36116235e-001,  2.05946350e-008,
        2.86524080e-001,  1.92623836e-001,  1.99023767e-001,
        3.69570465e-001,  7.40034063e-011,  1.88507265e-007,
        1.90129753e+001,  2.58588668e+000,  8.20012226e+001,
        7.53021801e-001,  4.59141850e-002,  6.96701067e-002,
        8.60737285e+000,  9.58185912e+000,  1.00964981e+001,
        6.79472273e+000,  7.77850482e+000,  3.00230730e+000,
        3.46535214e+000,  1.91385873e+000,  2.53437805e+000,
        2.58979566e-001,  3.26719230e-001,  3.16565244e-008,
        3.28993927e-002,  3.69114390e-002,  1.24167910e-001,
        3.04110518e-001,  3.28182995e-008,  3.58462331e-008,
        2.87975286e-005,  1.30272279e+001,  8.61928170e-002,
        6.20996534e+000,  3.67924881e+000,  1.66715658e-004,
        2.49415065e-023,  6.33969766e-001,  1.80801117e-011,
        5.56465695e-002,  6.08385375e-002,  6.57950206e+000,
        5.29482942e+000,  6.87842009e-033,  7.96564105e-011,
        8.33503849e-002,  5.42324939e-071,  3.10550188e-008,
        1.97822969e-006,  2.42981258e-017,  4.62685175e-104,
        7.03272873e-004,  7.94668728e-004,  1.47742493e-003,
        7.42015568e-004,  6.35427799e-070,  3.50897491e-006,
        5.21312642e-017,  3.07859231e-247,  2.58357082e-109,
        5.89661445e-004,  2.11616673e-004,  4.52619153e-004,
        1.71213881e-004,  4.84610805e-025,  2.46687204e-103,
        5.30636196e-015,  8.65750779e-012,  6.83302438e-116,
        1.04203889e-038,  5.52448150e-015,  1.39726440e-003,
        5.14035141e+000,  9.51452316e-004,  1.33365812e-003,
        1.06823103e-003,  4.31157829e-010,  4.27179136e-114,
        2.97856703e-028,  0.00000000e+000,  2.63345382e+000,
        2.78834644e-005,  8.34729323e-002,  0.00000000e+000,
        1.27354473e-003,  1.04381729e+001,  2.30427130e-003,
        2.84646084e+000,  8.85514140e-004,  4.47931328e+000,
        1.14885217e-004,  3.42363919e+000,  1.36532681e+001,
        1.12069191e-001,  2.47269338e-004,  4.84688808e-004,
        0.00000000e+000,  9.34471177e-001,  8.84910428e-024,
        1.87548975e-217,  6.67679702e-018,  2.85931610e-019,
        2.79641264e-020,  1.01165080e-184,  1.65322448e-015,
        4.41487672e-005,  1.42309455e-002,  5.93610104e-019,
        1.94592911e-124,  0.00000000e+000,  8.46374012e-062,
        0.00000000e+000,  5.86555998e-006,  9.72828140e-062,
        4.47663279e-105,  0.00000000e+000,  3.15431582e-092,
        0.00000000e+000,  3.51707333e-008,  0.00000000e+000,
        0.00000000e+000,  0.00000000e+000,  7.77815139e-078,
        7.44364292e-028,  7.64574663e-031,  2.19661735e-001,
        2.95822179e+000,  1.30655194e-001,  2.82124452e-028,
        2.88924640e-031,  2.45638971e-018,  4.26469187e-003,
        6.20529438e-078,  0.00000000e+000,  4.53656851e-011,
        9.95838225e-018,  3.05434950e-063,  1.11302353e-005,
        3.33169539e-002,  1.67364651e+000,  9.99141628e-028,
        1.12669156e-030,  1.09744458e-010,  2.18478693e-017,
        8.94566928e-006,  5.73128771e-006,  4.31864788e-051,
        3.24250875e-078,  6.00981790e-028,  0.00000000e+000,
        1.11722543e-017,  7.38263966e-003,  3.90465660e+000,
        3.89135007e-080,  1.02627747e-077,  1.18312935e-030,
        1.99477234e-017,  1.85843993e-008,  3.18997713e+000,
        3.21960342e-004,  7.04049630e-078,  5.43540001e-028,
        5.08243692e-031,  4.50057844e-011,  8.81497450e-071,
        1.27623698e-069,  3.69489015e-019,  7.28800816e-031,
        6.43694779e-011,  2.07685653e-063,  6.11422121e-009,
        3.23202800e-006,  1.16496389e-030,  0.00000000e+000,
        0.00000000e+000, -1.85218466e-177,  0.00000000e+000,
        0.00000000e+000,  2.51722038e-006,  3.01387771e-027])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge/RepeatedKFold/split_9/test_residuals_histogram.png'
stats = OrderedDict([('R2', 0.018889512167304834), ('root_mean_squared_error', 136.26261102873644), ('mean_absolute_error', 104.62829468730243), ('rmse_over_stdev', 1.4018867266661694)])
title = 'test_residuals_histogram'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_residuals_histogram(y_true, y_pred, savepath, stats, title, label)
display(Image(filename='test_residuals_histogram.png'))
